In [1]:
from IPython.display import display
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn
import pandas as pd
import time
import matplotlib.pyplot as plt
import matplotlib
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
matplotlib.rc('figure', figsize=[10,5])
import matplotlib.pyplot as plt
plt.style.use(['seaborn'])

hr_df = pd.read_csv("C:\\Users\\kelle\\Downloads\\HR_Data\\aug_train.csv")

def plot_learning_curve(train_scores, val_scores, indices, title, inverse_x=False):
    plt.plot(indices, train_scores , "g-+", linewidth=2, label="train")
    plt.plot(indices, val_scores, "b-+", linewidth=2, label="validation")
    plt.title(title)
    plt.legend()
    plt.xlabel('Number of Training Instances Used')
    plt.grid(color='black', linestyle='-', linewidth=0.5, alpha=0.3)
    if inverse_x:
        plt.gca().invert_xaxis()
        
def plot_complexity(train_scores, val_scores, indices, title, inverse_x=False):
    plt.plot(indices, train_scores , "g-+", linewidth=2, label="train")
    plt.plot(indices, val_scores, "b-+", linewidth=2, label="validation")
    plt.title(title)
    plt.legend()
    plt.grid(color='black', linestyle='-', linewidth=0.5, alpha=0.3)
    plt.gca().invert_xaxis()
    
import matplotlib.pyplot as plt
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0,1], [0,1], 'k--')
    plt.axis([0,1,0,1])
    plt.legend()
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone    

def stratFold(train_X_all, train_y_all, classifier, metric, average=None,
              fold=5):
    train_metric, val_metric, indices = [], [], []
    for m in range(100, len(train_X_all), 100):
        train_X = train_X_all[:m]
        train_y = train_y_all[:m]
        skfolds = StratifiedKFold(n_splits=fold)
        metric_list = []
        metric_list_train = []
        for train_index, test_index in skfolds.split(train_X, train_y):
            train_X_folds = train_X.iloc[train_index]
            train_y_folds = train_y.iloc[train_index]
            test_X_fold = train_X.iloc[test_index]
            test_y_fold = train_y.iloc[test_index]
            classifier.fit(train_X_folds.values, train_y_folds.values.ravel())
            train_y_folds_pred = classifier.predict(train_X_folds)
            test_y_fold_pred = classifier.predict(test_X_fold)
            if average:
                metric_list.append(metric(test_y_fold.values, test_y_fold_pred,
                                     average=average))
                metric_list_train.append(metric(train_y_folds.values, train_y_folds_pred, 
                                            average=average))
            else:
                metric_list.append(metric(test_y_fold.values, test_y_fold_pred))
                metric_list_train.append(metric(train_y_folds.values, train_y_folds_pred))
                
        test_avg = sum(metric_list)/len(metric_list)
        train_avg = sum(metric_list_train)/len(metric_list_train)
        val_metric.append(test_avg)
        train_metric.append(train_avg)
        indices.append(m)
    
    return train_metric, val_metric, indices

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

hr_df['city']=hr_df['city'].astype(str)
hr_df['gender']=hr_df['gender'].astype(str)
hr_df['relevent_experience']=hr_df['relevent_experience'].astype(str)
hr_df['enrolled_university']=hr_df['enrolled_university'].astype(str)
hr_df['education_level']=hr_df['education_level'].astype(str)
hr_df['major_discipline']=hr_df['major_discipline'].astype(str)
hr_df['experience']=hr_df['experience'].astype(str)
hr_df['company_size']=hr_df['company_size'].astype(str)
hr_df['company_type']=hr_df['company_type'].astype(str)
hr_df['last_new_job']=hr_df['last_new_job'].astype(str)
hr_df = MultiColumnLabelEncoder(columns = ['city','gender','relevent_experience','enrolled_university','education_level','major_discipline','experience','company_size','company_type','last_new_job']).fit_transform(hr_df)



from sklearn.model_selection import StratifiedShuffleSplit
data_split = StratifiedShuffleSplit(n_splits=1 , test_size=0.3, random_state=30)
for train_ind, test_ind in data_split.split(hr_df, hr_df["target"]):
    strat_train_set = hr_df.loc[train_ind]
    strat_test_set = hr_df.loc[test_ind]
    
train_set = strat_train_set
test_set = strat_test_set

train_y = train_set[["target"]]
train_X = train_set.drop("target", axis=1)
test_y = test_set[["target"]]
test_X = test_set.drop("target", axis=1)

In [2]:
from sklearn.svm import SVC
hr_svm_classifier = SVC(gamma='auto', class_weight='balanced')
hr_svm_classifier.fit(train_X, train_y.values.ravel())

SVC(class_weight='balanced', gamma='auto')

In [ ]:
start = time.time()


from sklearn.metrics import f1_score
f1_scores = stratFold(train_X, train_y, hr_svm_classifier, f1_score, average="macro")
plot_learning_curve(f1_scores[0], f1_scores[1], f1_scores[2], 
                    title="F1 MACRO score learning curve ")

elapsed = (time.time() - start)

print(elapsed)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(hr_svm_classifier, train_X, train_y.values.ravel(),
                         scoring="f1_macro", cv=10)
scores.mean()

In [ ]:
from sklearn.model_selection import validation_curve
hr_svm = SVC(C=4.5, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.0016025641025641025,
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

param_range = np.linspace(1, 500, 100)

train_scores, test_scores = validation_curve(
    hr_svm, train_X, train_y.values.ravel(), param_name="C", 
    param_range=param_range, scoring='f1_macro', verbose=1, cv=5, n_jobs=-1
)

print(train_scores.mean(axis=1), test_scores.mean(axis=1), param_range)
plot_complexity(train_scores.mean(axis=1), test_scores.mean(axis=1), param_range, 
                title='ROC Complexity Curve on ccp_alpha', inverse_x=False)